# Predicting in-hospital mortality

As we discussed in our session yesterday, we want to be able to predict in-hospital mortality since it helps us communicate bette the risks and benefits to patients, families, and caregivers. For this exercise we will be using the same dataset we used for the exploratory data analysis excercisr from a couple of days ago.

As always, we start by importing the packages that we will be using

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Now we are going to read the data. It is divided into two csv files, one with the patient attributes and the other one with the outcomes:

In [ ]:
dataset = pd.read_csv("/home/shared/icu_2012.txt")
outcomes = pd.read_csv("/home/shared/Outcomes-a.txt")

Let's now see how this looks like:

In [ ]:
dataset.head()

In [ ]:
outcomes.head(20)

We will be using the `In-hospital_death` label. You can see that it also has other variables like Length of stay and severity scores (SAPS-I, SOFA) which now you know what they are.

For practical reasons, I want to merge these two datasets so we will do a left join. This means that we wil 'stick' the outcomes labels and add them to the dataset with patient attributes. That is what the `pd.merge` method does, it matches according to `Record_ID`.

I'll also change the name of `In-hospital_death`, I need to get rid of the dash so I'll change it for an underscore:

In [ ]:
tree_dataset = pd.merge(dataset, outcomes, on='RecordID', how='left')
tree_dataset.rename(columns={'In-hospital_death': 'In_hospital_death'}, inplace=True)
tree_dataset.head()

Yo can scroll right and see that the columns have been added. Just like we did with our exploratory data analysis, we will take a look at our dataset:

In [ ]:
tree_dataset.columns

To make things a bit easier for us we will work with a subset of the variables. Here I arbitrarily decided to use the ones that make up the SOFA score. Usually this process of selecting features should be guided by clinical knowledge and the data that you are working with. I will go ahead and change my `tree_dataset` so it only has the columns that I want:

In [ ]:
tree_dataset = tree_dataset[['Age', 'Bilirubin_max', 'Creatinine_max', 'FiO2_max', 'GCS_min', 'MAP_min', 'Platelets_min', 'In_hospital_death']]
tree_dataset.head()

In [ ]:
tree_dataset.shape

The classifier that we will be training doesn't like categorical variables not missing values. That is something that you need to work with. In this case we don't have any categorical variables so that is not a problem. If that were the case, we could use [`pd.get_dummies()` method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html).

We do have quite some missing values. One thing you can do is impute missing data. Statisticians devote entire lifetimes of research to find better ways to [impute missing data](https://en.wikipedia.org/wiki/Imputation_%28statistics%29) (we will not do that here). I will fill the missing values with the last valid value in that column using the method `fillna` with the `backfill` option. There are many other (probably better) ways of doing this, this is just an example.


In [ ]:
tree_dataset.fillna(method='backfill', inplace = True)

I will check if there are still missing values:

In [ ]:
tree_dataset.isnull().any()

Bilirrubin still has missing value because thhe `backfill` method only checks a number of columns down. I will go afead and replace the missing values with normal bilirrubins (1).

In [ ]:
tree_dataset[['Bilirubin_max']] = tree_dataset[['Bilirubin_max']].fillna(value=1)

In [ ]:
tree_dataset.isnull().any()

## Decision Trees

Now that we are ready, we will train our decision tree. But first we need to make sure we know where we are storing the patient data and the labels (in_hospital_death). We will create two new datasets:

X will store all the patient variables
y will store all the true labels

Of course we could have worked with the two separate datasets that we had at the beginning!!


In [ ]:
# Here I am dropping the label, so X will have all the patient data 
X = tree_dataset.drop('In_hospital_death', axis=1)
# Here I am setting y as only the labels
y = tree_dataset['In_hospital_death']

In [ ]:
X.head()

In [ ]:
y.head()

Now we will divide our data in two. One sample of patients will be the **TRAINING SET**, the data I will use to train the model, the other sample will be the **TESTING SET** the data I will use to test gow good (or bad) is my model.

To do this, I will import the necessary packages from scikit:

In [ ]:
from sklearn.model_selection import train_test_split

Here what I am doing is the following:

`X_train`: contains all patient attributes from the patients I'll use to train the model.

`X_test`: contains all patient attributes from the patients I'll use to test the model. The model will not see this patients when I train it.

Same thing for `y_train` and `y_test`

And in the parameters, I am giving the function three elements: X (where all may patient data lives), y (where all the labels live), and `test_size` which defines the size of the training/testing sets. In this case is 80% training, 20% testing 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
X_test.count()

We can see that  `X_test` has 800 patients (20% of the original 4000) :)

We will now import the packages we need to actually train the classifier. In this case we need a DecisionTreeClassifier. 

In [ ]:
# We import the package
from sklearn.tree import DecisionTreeClassifier

# we create an instance of a DecisionTreeClassifier that we will name, creatively, classifier:
classifier = DecisionTreeClassifier()

# now we ask the classifier to learn from the training dataset:
classifier.fit(X_train, y_train)

In the cell above we can see that we didn't sepcify any specific variable for the decision tree so it used all the defaults. We will change some of that later. You can look at all the options [here](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)

Now we need to check how we did. For that, we will ask the classifier to predict the labels, given a set of patient attributes. We use the `predict` method and we pass it the `X_test` data that contains all the patient attributes that the classifier has not seen. We will store the predictions in `y_pred`

In [ ]:
y_pred = classifier.predict(X_test)

Mindblowing!!! not really, we don't see anything happen. 

We now need to calculate some performance metrics. To do this we will use a confusion matrix and a classification report: 

In [ ]:
#we import what we need
from sklearn.metrics import classification_report, confusion_matrix

We compare the classifier's predictions stored in `y_pred` with the truth that is stored in `y_test`

In [ ]:

print(confusion_matrix(y_test, y_pred))

<table style="width:20%">
<tr>
<th></th>
<th>Survival Pred</th>
<th>Death Pred</th>
<th>TOTAL</th>
</tr>

<tr>
<td>True Survival</td>
<td>581</td>
<td>104</td>
<td>685</td>
</tr>

<tr>
<td>True Death</td>
<td>77</td>
<td>38</td>
<td>115</td>
</tr>
</table>

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
pd.options.display.max_columns = None
tree_dataset.columns

In [ ]:
feature_cols = ['Age', 'Bilirubin_max', 'Creatinine_max', 'FiO2_max', 'GCS_min',
       'MAP_min', 'Platelets_min']

One of the cool things about decision trees is that they are interpretable. I can understand how did are they built and what is guiding the decisions. Here we will visualize the decision tree we just built:

In [ ]:
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus
dot_data = StringIO()
export_graphviz(classifier, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,feature_names = feature_cols,class_names=['0','1'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

That is not very interpretable!! Since we did not set any option, it defaulted to creating a tree with unlimited depth, and that is what we got. Let's try 'pruning' the tree with a maximum depth of 6 so we can make a better interpretation, and let's see if we lose accuracy in the process:

In [ ]:
#from sklearn.tree import DecisionTreeClassifier
classifier2 = DecisionTreeClassifier(max_depth = 6)
classifier2.fit(X_train, y_train)

y_pred2 = classifier2.predict(X_test)


from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred2))
print(classification_report(y_test, y_pred2))

Not bad! Our accuracy even went up a bit (from 77 to 84%). Let's see how the tree looks like:

In [ ]:
dot_data = StringIO()
export_graphviz(classifier2, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,feature_names = feature_cols,class_names=['0','1'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

A bit better, but still not very intepretable. Let's try with Max depth = 3

In [ ]:
classifier3 = DecisionTreeClassifier(max_depth = 3)
classifier3.fit(X_train, y_train)

y_pred3 = classifier3.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred3))
print(classification_report(y_test, y_pred3))

Looking good! Our accuracy is pretty much the same but now we might have a model that we can interpret better. Let's see:

In [ ]:
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus
dot_data = StringIO()
export_graphviz(classifier3, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,feature_names = feature_cols,class_names=['0','1'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

Now you can really interpret what the tree is doing to classify patients. However, we can see that is only predicting Death(+) in patients with ALL the following conditions: Creatinine > 1.35, Platelets < 86,500 and Bilirrubin >5.6.

That is pretty narrow. It is getting away with reasonable accuracy because mortality in our dataset is low, also know as **Class Imbalance**. Since only 13% of all patients die, if the model predicts **survival** for all patients, it will be right 87% of the time... which is what is happening here. We will deal with that a bit later. 

In [ ]:
tree_dataset.In_hospital_death.value_counts()

## Now we will calculate AUC

In [ ]:
prob_y = classifier3.predict_proba(X)

In [ ]:
prob_y = [p[1] for p in prob_y]

In [ ]:
from sklearn.metrics import roc_auc_score
print( roc_auc_score(y, prob_y) )

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

false_positive_rate, true_positive_rate, thresholds = roc_curve(y, prob_y)
roc_auc = auc(false_positive_rate, true_positive_rate)

plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate, 'b',
label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.0,1.0])
plt.ylim([-0.0,1.0])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

## Now we will try to fix the unbalanced class problem

To acomplish this we will separate the dataset into those who died and those who survived:

In [ ]:
	
from sklearn.utils import resample

# Separate majority and minority classes
df_majority = tree_dataset[tree_dataset.In_hospital_death==0]
df_minority = tree_dataset[tree_dataset.In_hospital_death==1]

In [ ]:
#let's see how it looks
df_majority.head()

In [ ]:
# and now the minority
df_minority.head()

We will now upsample the death cases using the Pandas `resample()` method. Just as in imputation, there are MANY ways of doing this and some are smarter than this one, scikit has very smart methods of doing this too. We will not spend too much time doing that.

In [ ]:
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=3446,    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
 
# Display new class counts
df_upsampled.In_hospital_death.value_counts()

In [ ]:
y = df_upsampled.In_hospital_death
X = df_upsampled.drop('In_hospital_death', axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
classifier4 = DecisionTreeClassifier(max_depth = 3)
classifier4.fit(X_train, y_train)

In [ ]:
y_pred4 = classifier4.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred4))
print(classification_report(y_test, y_pred4))

In [ ]:
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus
dot_data = StringIO()
export_graphviz(classifier4, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,feature_names = feature_cols,class_names=['0','1'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

In [ ]:
prob_y = classifier4.predict_proba(X)

In [ ]:
prob_y = [p[1] for p in prob_y]

In [ ]:
from sklearn.metrics import roc_auc_score
print( roc_auc_score(y, prob_y) )

In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(y, prob_y)
roc_auc = auc(false_positive_rate, true_positive_rate)

plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate, 'b',
label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.0,1.0])
plt.ylim([-0.0,1.0])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

### There has to be a better way

We will finally try to approach this problem with what we call an ensemble, that is a combination of machine learning algorithms. In this case we will use a Random Forest. As we discussed, a random forest is a combination of randomly generated decision trees trained with a random sample of the data. This tends to produce way better results than a single decision tree, in particular when we have unbalanced classes (like this example). Let's see:  

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

Note that we are using the original data, without upsampling.

In [ ]:
##### Separate input features (X) and target variable (y)
y = tree_dataset.In_hospital_death
X = tree_dataset.drop('In_hospital_death', axis=1)
 
# Train model
clf_5 = RandomForestClassifier(max_depth=8)
clf_5.fit(X, y)
 
# Predict on training set
pred_y_5 = clf_5.predict(X)

In [ ]:
# Is our model still predicting just one class?
print( np.unique( pred_y_5 ) )

In [ ]:
predictions = pd.DataFrame(pred_y_5)
predictions.describe()

In [ ]:
# How's our accuracy?
print( accuracy_score(y, pred_y_5) )

In [ ]:
# What about AUROC?
prob_y_5 = clf_5.predict_proba(X)

prob_y_5 = [p[1] for p in prob_y_5]

print( roc_auc_score(y, prob_y_5) )

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

false_positive_rate, true_positive_rate, thresholds = roc_curve(y, prob_y_5)
roc_auc = auc(false_positive_rate, true_positive_rate)

plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate, 'b',
label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.0,1.0])
plt.ylim([-0.0,1.0])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()